# Using Parallel Magics

IPython has a few magics for working with your engines.

This assumes you have started an IPython cluster, either with the notebook interface,
or the `ipcluster/controller/engine` commands.

In [ ]:
from IPython import parallel
rc = parallel.Client()
dv = rc[:]
rc.ids

Creating a Client registers the parallel magics `%px`, `%%px`, `%pxresult`, `pxconfig`, and `%autopx`.  
These magics are initially associated with a DirectView always associated with all currently registered engines.

Now we can execute code remotely with `%px`:

In [ ]:
%px a=5

In [ ]:
%px print(a)

In [ ]:
%px a

In [ ]:
with dv.sync_imports():
    import sys

In [ ]:
%px from __future__ import print_function
%px print("ERROR", file=sys.stderr)

You don't have to wait for results.  The `%pxconfig` magic lets you change the default blocking/targets for the `%px` magics:

In [ ]:
%pxconfig --noblock

In [ ]:
%px import time
%px time.sleep(5)
%px time.time()

But you will notice that this didn't output the result of the last command.
For this, we have `%pxresult`, which displays the output of the latest request:

In [ ]:
%pxresult

Remember, an IPython engine is IPython, so you can do magics remotely as well!

In [ ]:
%%px
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

`%%px` can also be used as a cell magic, for submitting whole blocks.
This one acceps `--block` and `--noblock` flags to specify
the blocking behavior, though the default is unchanged.


In [ ]:
dv.scatter('id', dv.targets, flatten=True)
dv['stride'] = len(dv)

In [ ]:
%%px --block
print("    id=%s" % id)
print("stride=%s" % stride)

In [ ]:
%%px --noblock
x = np.linspace(0,np.pi,1000)
for n in range(id, 12, stride):
    print(n)
    plt.plot(x,np.sin(n*x))
plt.title("Plot %i" % id);

In [ ]:
%pxresult

## Parallel Exceptions

When you raise exceptions with the parallel exception,
the CompositeError raised locally will display your remote traceback.

In [ ]:
%%px
from numpy.random import random
A = random((100,100,'invalid shape'))

## Remote Cell Magics

Remember, Engines are IPython too, so the cell that is run remotely by %%px can in turn use a cell magic.

In [ ]:
%%px
%%bash
echo "remote parallel interactive bash!"
hostname
date
echo $$

In [ ]:
%%px
%%ruby
puts 'hello from ruby'

In [ ]:
dv.scatter('rank', dv.targets, flatten=True)

In [ ]:
%%px
%%timeit
from numpy.random import random
from numpy.linalg import norm
N = 100 * (rank + 1)
A = random((N,N))
norm(A, 2)    

# Debugging Engines

Since the IPython engine is precisely the same object used for the notebook and qtconsole,
we can connect other fronteds directly to the engine.

The first step is to bind the engine's sockets, so its connection pattern looks like a regular kernel

In [ ]:
%%px
from IPython.parallel import bind_kernel
bind_kernel()

Now we can raise an exception on the engines

In [ ]:
%%px

def foo(a, b):
    return a/(1-b)

def bar(b):
    return foo(2, b)

bar(1)

Now we can connect a qtconsole to the engine(s)

In [ ]:
%px %qtconsole